**Extract - Transform - Load**

Import neccesary libraries

In [1]:
import pandas as pd
import simfin as sf
from dotenv import load_dotenv
import os
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, classification_report
import pickle
import numpy as np

Load the CSV Files

In [2]:
load_dotenv()

# Get API key and data directory from environment variables
api_key = os.getenv("SIMFIN_API_KEY")
data_dir = os.getenv("SIMFIN_DATA_DIR")

# Set API key and data directory for SimFin
sf.set_api_key(api_key)
sf.set_data_dir(data_dir)

# Download the data from SimFin and load into a Pandas DataFrame
df_companies = sf.load_companies(market='us')
df_shareprices = sf.load_shareprices(variant='daily')

Dataset "us-companies" on disk (15 days old).
- Loading from disk ... Done!
Dataset "us-shareprices-daily" on disk (15 days old).
- Loading from disk ... 

/Users/alejandroperez/.pyenv/versions/3.10.10/lib/python3.10/site-packages/simfin/load.py:154: FutureWarning: The argument 'date_parser' is deprecated and will be removed in a future version. Please use 'date_format' instead, or read your data in as 'object' dtype and then call 'to_datetime'.
  df = pd.read_csv(path, sep=';', header=0,
/Users/alejandroperez/.pyenv/versions/3.10.10/lib/python3.10/site-packages/simfin/load.py:154: FutureWarning: The argument 'date_parser' is deprecated and will be removed in a future version. Please use 'date_format' instead, or read your data in as 'object' dtype and then call 'to_datetime'.
  df = pd.read_csv(path, sep=';', header=0,


Done!


In [3]:
# Going with bulk download (local) for now to see structure difference

# Assigning it again with separator to fix structure issues

df_companies = pd.read_csv('/Users/alejandroperez/Desktop/Python_II_Final_Project/~datasets/us-companies.csv', sep=";")
df_shareprices = pd.read_csv('/Users/alejandroperez/Desktop/Python_II_Final_Project/~datasets/us-shareprices-daily.csv', sep=";")

In [ ]:
# Define the tickers we want to focus on - Apple, Microsoft, Amazon, Tesla, and Meta

top_tickers = ['AAPL', 'MSFT', 'AMZN', 'TSLA', 'META']

In [5]:
# Filter for tickers of choice

df_companies = df_companies[df_companies['Ticker'].isin(top_tickers)]
df_shareprices = df_shareprices[df_shareprices['Ticker'].isin(top_tickers)]

Inspect dataframes

In [6]:
print("Companies DataFrame Head:")
df_companies.head()

Companies DataFrame Head:


,Ticker,SimFinId,Company Name,IndustryId,ISIN,End of financial year (month),Number Employees,Business Summary,Market,CIK,Main Currency
90,AAPL,111052,APPLE INC,101001.0,US0378331005,9.0,147000.0,Apple Inc is an American multinational technol...,us,320193.0,USD
412,AMZN,62747,AMAZON COM INC,103002.0,US0231351067,12.0,1298000.0,Amazon.com Inc is an online retailer. The Comp...,us,1018724.0,USD
3505,META,121021,"Meta Platforms, Inc.",101002.0,US30303M1027,12.0,58604.0,Facebook Inc is the world\'s largest online so...,us,1326801.0,USD
3663,MSFT,59265,MICROSOFT CORP,101003.0,US5949181045,6.0,166475.0,Microsoft Corp is a technology company. It dev...,us,789019.0,USD
5492,TSLA,56317,Tesla,103005.0,US88160R1014,12.0,70757.0,"Tesla Motors, Inc., incorporated on July 1, 20...",us,1318605.0,USD


In [7]:
print("Shareprices DataFrame Head:")
df_shareprices.head()

Shareprices DataFrame Head:


,Ticker,SimFinId,Date,Open,High,Low,Close,Adj. Close,Volume,Dividend,Shares Outstanding
14269,AAPL,111052,2019-04-08,49.10,50.06,49.09,50.02,47.93,103526788,NaN,1.842914e+10
14270,AAPL,111052,2019-04-09,50.08,50.71,49.81,49.88,47.79,143072948,NaN,1.842914e+10
14271,AAPL,111052,2019-04-10,49.67,50.19,49.55,50.16,48.06,86781152,NaN,1.842914e+10
14272,AAPL,111052,2019-04-11,50.21,50.25,49.61,49.74,47.66,83603232,NaN,1.842914e+10
14273,AAPL,111052,2019-04-12,49.80,50.03,49.05,49.72,47.64,111042672,NaN,1.842914e+10


In [8]:
print("\nCompanies DataFrame Missing Values:")
df_companies.isnull().sum()


Companies DataFrame Missing Values:


Ticker                           0
SimFinId                         0
Company Name                     0
IndustryId                       0
ISIN                             0
End of financial year (month)    0
Number Employees                 0
Business Summary                 0
Market                           0
CIK                              0
Main Currency                    0
dtype: int64

In [9]:
print("\nCompanies DataFrame Data Types:")
df_companies.dtypes


Companies DataFrame Data Types:


Ticker                            object
SimFinId                           int64
Company Name                      object
IndustryId                       float64
ISIN                              object
End of financial year (month)    float64
Number Employees                 float64
Business Summary                  object
Market                            object
CIK                              float64
Main Currency                     object
dtype: object

In [10]:
print("\nShare Prices DataFrame Missing Values:")
df_shareprices.isnull().sum()


Share Prices DataFrame Missing Values:


Ticker                   0
SimFinId                 0
Date                     0
Open                     0
High                     0
Low                      0
Close                    0
Adj. Close               0
Volume                   0
Dividend              6159
Shares Outstanding       0
dtype: int64

We see that Dividend has 6159 missing values (a lot) and we decide to drop it as some tech companies don't even give out dividends.

In [11]:
df_shareprices = df_shareprices.drop(columns=['Dividend'])
df_shareprices.isnull().sum()

Ticker                0
SimFinId              0
Date                  0
Open                  0
High                  0
Low                   0
Close                 0
Adj. Close            0
Volume                0
Shares Outstanding    0
dtype: int64

In [12]:
print("\nShare Prices DataFrame Data Types:")
df_shareprices.dtypes


Share Prices DataFrame Data Types:


Ticker                 object
SimFinId                int64
Date                   object
Open                  float64
High                  float64
Low                   float64
Close                 float64
Adj. Close            float64
Volume                  int64
Shares Outstanding    float64
dtype: object

We see Date is an object, so we change it to datetime

In [13]:
df_shareprices['Date'] = pd.to_datetime(df_shareprices['Date'])
print(df_shareprices.dtypes)  # Check if "Date" is now datetime64

Ticker                        object
SimFinId                       int64
Date                  datetime64[ns]
Open                         float64
High                         float64
Low                          float64
Close                        float64
Adj. Close                   float64
Volume                         int64
Shares Outstanding           float64
dtype: object


In [14]:
df_merged = pd.merge(df_shareprices, df_companies, on='Ticker', how='left', 
                     suffixes=('_price', '_company'))
print("\nMerged DataFrame Head:")
df_merged.head()


Merged DataFrame Head:


,Ticker,SimFinId_price,Date,Open,High,Low,Close,Adj. Close,Volume,Shares Outstanding,SimFinId_company,Company Name,IndustryId,ISIN,End of financial year (month),Number Employees,Business Summary,Market,CIK,Main Currency
0,AAPL,111052,2019-04-08,49.10,50.06,49.09,50.02,47.93,103526788,1.842914e+10,111052,APPLE INC,101001.0,US0378331005,9.0,147000.0,Apple Inc is an American multinational technol...,us,320193.0,USD
1,AAPL,111052,2019-04-09,50.08,50.71,49.81,49.88,47.79,143072948,1.842914e+10,111052,APPLE INC,101001.0,US0378331005,9.0,147000.0,Apple Inc is an American multinational technol...,us,320193.0,USD
2,AAPL,111052,2019-04-10,49.67,50.19,49.55,50.16,48.06,86781152,1.842914e+10,111052,APPLE INC,101001.0,US0378331005,9.0,147000.0,Apple Inc is an American multinational technol...,us,320193.0,USD
3,AAPL,111052,2019-04-11,50.21,50.25,49.61,49.74,47.66,83603232,1.842914e+10,111052,APPLE INC,101001.0,US0378331005,9.0,147000.0,Apple Inc is an American multinational technol...,us,320193.0,USD
4,AAPL,111052,2019-04-12,49.80,50.03,49.05,49.72,47.64,111042672,1.842914e+10,111052,APPLE INC,101001.0,US0378331005,9.0,147000.0,Apple Inc is an American multinational technol...,us,320193.0,USD


In [15]:
output_file = '/Users/alejandroperez/Desktop/Python_II_Final_Project/~datasets/merged_data.csv'

# Save the merged dataframe as a CSV file without the index.
df_merged.to_csv(output_file, index=False)
print(f"Merged DataFrame saved to {output_file}")


Merged DataFrame saved to /Users/alejandroperez/Desktop/Python_II_Final_Project/~datasets/merged_data.csv
